In [19]:
from transformers import BertTokenizer, BertModel
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
model.eval();

In [21]:
import pandas as pd

In [22]:
year = 2015

In [23]:
df = pd.read_csv('../articles_raw_data/news_' + str(year) + '.csv')

In [24]:
df = df['text']

In [25]:
bag = [item for sentence in df for item in sentence.split('.') if item != '']
bag_size = len(bag)

In [26]:
bag_size

660027

In [27]:
targets = ['abuse', 'administration', 'cold', 'wall', 'aids', 'authority', 'battle', 'black', 'campaign', 'lives', 'head', 'march', 'public', 'release', 'screen', 'shot', 'war', 'union', 'white' ]

In [28]:
len(targets)

19

In [29]:
sentences = []


for i in targets:
    count = 0
    print(i)
    for j in range(bag_size):

        if count < 1000:
            sentence = bag[j].split()

            if len(sentence) > 512:
                sentence = sentence[:512]


            if i in sentence:
                sentences.append(bag[j])
                count += 1
            else:
                continue
        else:
            break
    
    

print(len(sentences))

abuse
administration
cold
wall
aids
authority
battle
black
campaign
lives
head
march
public
release
screen
shot
war
union
white
17717


In [30]:
file = open('../articles_raw_data/' + str(year) + '_sentences.txt','w') 

for item in sentences:
	file.write(item+"\n")
file.close()

In [31]:
def infer_vector(doc:str):

    marked_text = "[CLS] " + doc + " [SEP]"
    tokens = bert_tokenizer.tokenize(marked_text)[:512]
    idx = bert_tokenizer.convert_tokens_to_ids(tokens)
    segment_id = [1] * len(tokens)


    tokens_tensor = torch.tensor([idx])
    segments_tensors = torch.tensor([segment_id])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    hidden_states = hidden_states

    return hidden_states[-2][0], tokens

In [32]:
len(sentences)

17717

In [33]:
results = {k: {'sentence_number_index': [] , 'embeddings': []} for k in targets}

for i in range(len(sentences)): #len(sentences)
    if i%1000 == 0:
        print(i)

    sentence = sentences[i].split()[:250]

    for word in targets:

        if word in sentence:
            embeddings, tokens = infer_vector(sentences[i])

            index = tokens.index(word)
            embedding = embeddings[index].tolist()

            results[word]['sentence_number_index'].append([i, index])
            results[word]['embeddings'].append(embedding)
        
        else:
            continue




0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [34]:
file = []

for word in targets:
    results[word]['word'] = word
    file.append(results[word])

In [35]:
import json

output_file = open('../embeddings/embeddings_' + str(year) + '.json', 'w', encoding='utf-8')
for dic in file:
    
    json.dump(dic, output_file) 
    output_file.write("\n")

In [36]:
print('../embeddings/embeddings_' + str(year) + '.json')

../embeddings/embeddings_2015.json


# Done